In [293]:
import plotly.plotly as py
import pandas as pd
import os
import random
from pylab import *
import matplotlib.pyplot as plt

In [268]:
plotly.tools.set_credentials_file(username='CANada-2018', api_key='qxDFj32iznKNE8t5VuUv')

In [90]:
reading_folder = "./visualization/"
file_list = [i for i in sorted(os.listdir(reading_folder)) if '.csv' in i]

In [122]:
ONE_MILLION = 1000000
ONE_100_K = 100000

In [60]:
DALY_dict = dict.fromkeys(file_list)
for file in file_list:
    print("Reading {} ...".format(file))
    DALY_dict[file] = pd.read_csv(reading_folder + file, index_col=0)

Reading DALY_Africa_Number.csv ...
Reading DALY_Africa_Rate.csv ...
Reading DALY_America_Number.csv ...
Reading DALY_America_Rate.csv ...
Reading DALY_Asia_Number.csv ...
Reading DALY_Asia_Rate.csv ...
Reading DALY_Australia_Number.csv ...
Reading DALY_Australia_Rate.csv ...
Reading DALY_China_Number.csv ...
Reading DALY_China_Rate.csv ...
Reading DALY_Europe_Number.csv ...
Reading DALY_Europe_Rate.csv ...
Reading DALY_France_Number.csv ...
Reading DALY_France_Rate.csv ...
Reading DALY_Global_Number.csv ...
Reading DALY_Global_Rate.csv ...
Reading DALY_Number.csv ...
Reading DALY_Oceania_Number.csv ...
Reading DALY_Oceania_Rate.csv ...
Reading DALY_Rate.csv ...
Reading DALY_Russian Federation_Number.csv ...
Reading DALY_Russian Federation_Rate.csv ...
Reading DALY_United States_Number.csv ...
Reading DALY_United States_Rate.csv ...


In [318]:
DALY_number = DALY_dict['DALY_Number.csv']
DALY_rate = DALY_dict['DALY_Rate.csv']

# Categorize the countries
glob = ['Global']
continents = ['Asia', 'Europe', 'Oceania', 'America', 'Africa']
countries = [ i for i in sorted(DALY_number.location.unique()) if ((not i in continents) and (i != 'Global'))]

level_1 = ['Mental disorders']
depression = ['Major depressive disorder', 'Dysthymia']
eating = ['Anorexia nervosa', 'Bulimia nervosa']
level_2 = [i for i in DALY_number.cause.unique() if (i not in level_1) and (i not in depression) and (i not in eating) ]

location = continents
df = DALY_number
sex = 'Both'
disorder = level_2
age = 'All Ages'

df_year = df[(df.year == 2017) & (df.location.isin(location)) & (df.cause.isin(disorder)) & (df.sex == sex) & (df.age == age)]

# Labels for the graph
labels = sorted(list(df_year.cause.unique()))

# Colors for each node
cmap = cm.get_cmap('viridis', len(labels))    # PiYG
colors = []
rgbas = []
for i in range(cmap.N):
    rgb = cmap(i)[:3] # will return rgba, we take only first 3 so we get rgb
    colors.append(matplotlib.colors.rgb2hex(rgb))
    rgbas.append(rgb + (0.5,))

labels.extend(sorted(location))

cmap = cm.get_cmap('Reds', len(location))    # PiYG
for i in range(cmap.N):
    rgb = cmap(i)[:3] # will return rgba, we take only first 3 so we get rgb
    colors.append(matplotlib.colors.rgb2hex(rgb))


df_location = df_year.groupby(['cause', 'location'])['val'].sum().unstack()
# Percentage per mental illness
df_disorder_std = df_location.apply(lambda x: x/x.sum() * 100)
# Percentage per location
df_location_std = df_location.apply(lambda x: x/x.sum() * 100, axis=1)

# Source - Locations
source = []
for j in range(len(df_location.columns)):
    source.extend([i for i in range(len(df_location.index))])

# Target - Mental illness
target = []
for j in range(len(df_location.columns)):
    target.extend([j + len(df_location.index) for i in range(len(df_location.index))])
    
# Values
values = []
for i in df_location.columns:
    values.extend(list(df_location[i]))

# Link colors
link_colors = []
for i in source:
    link_colors.append("rgba{}".format(rgbas[i]))

In [324]:
data = dict(
    type='sankey',
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(
        color = "black",
        width = 0.5
      ),
      label = labels,
      color = colors
    ),
    orientation = "h",
    valueformat = ".0f",
    link = dict(
      source = source,
      target = target,
      value = values,
      color = link_colors
  ))

layout =  dict(
    title = "Disability-Adjusted Life Years (DALY) per Mental Illness on a continent level",
    font = dict(
      size = 10
    )
)

fig = dict(data=[data], layout=layout)
py.iplot(fig, validate=False)